In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
print(llm)

Ollama
Params: {'model': 'llama2', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None}


In [3]:
llm.invoke("how can langsmith help with testing?")

"\nLangSmith is a tool that helps with natural language processing (NLP) tasks, including text testing. Here are some ways LangSmith can assist with testing:\n\n1. Sentiment Analysis: LangSmith's sentiment analysis feature can help identify the emotions and opinions expressed in a piece of text. This can be useful for testing the tone and sentiment of content, such as customer reviews or social media posts.\n2. Topic Modeling: LangSmith's topic modeling feature can help identify the underlying topics or themes in a large corpus of text data. This can be useful for testing the consistency and coherence of content across multiple documents or websites.\n3. Named Entity Recognition (NER): LangSmith's NER feature can help identify and classify named entities in text, such as people, organizations, and locations. This can be useful for testing the accuracy and completeness of information in a piece of text.\n4. Part-of-Speech (POS) Tagging: LangSmith's POS tagging feature can help identify 

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LanguSmith is an incredible tool for any technical writer, especially when it comes to testing. Here are some ways in which LanguSmith can help with testing:\n\n1. Automated Translation Testing: With LanguSmith, you can easily translate your content into multiple languages and test its accuracy using automated translation tools. This can save you a significant amount of time and effort compared to manually translating and testing each language version separately.\n2. Language Review and Correction: LanguSmith's language review feature allows you to quickly check the quality and accuracy of your content in multiple languages. You can use this feature to identify any errors or inconsistencies in the translated content, and make necessary corrections before publishing.\n3. Cross-Cultural Review: With LanguSmith, you can perform cross-cultural reviews of your content to ensure that it is appropriate and culturally relevant

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [6]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a powerful tool that can greatly assist in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Testing: With Langsmith, you can create automated tests for your software applications, websites, or systems. The platform allows you to write and run tests using natural language queries, making it easier to test complex scenarios and edge cases.\n2. Smart Test Selection: Langsmith's AI-powered test selection feature helps identify the most relevant tests based on your requirements, saving time and effort. You can easily focus on the most important tests and optimize your testing strategy.\n3. Contextual Help: Langsmith's AI-driven contextual help provides instant support to users during testing. The platform offers intelligent suggestions and recommendations based on the user's input, helping them navigate complex test scenarios and resolve issues more efficiently.\n4. Integration with C

In [7]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by allowing users to visualize test results.'

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. **Creating test cases**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, that can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
2. **Evaluating performance**: LangSmith provides an ability to score test results using custom evaluations (both LLM and heuristic-based). This helps developers to track regressions with respect to their initial test cases and diagnose issues in their applications.
3. **Debugging issues**: When things go wrong (e.g., unexpected end result, infinite agent loop, slower than expected execution), LangSmith gives clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.
4. **Sending runs to annotation queues**: Annotators can closely inspect interesting trace

In [14]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [15]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [17]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug wh

In [18]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

In [19]:
tool = PubmedQueryRun()

In [20]:
tool.invoke("What causes lung cancer?")

"Published: 2024-05-05\nTitle: CLIP170 inhibits the metastasis and EMT of papillary thyroid cancer through the TGF-β pathway.\nCopyright Information: © 2024. The Author(s), under exclusive licence to Springer Science+Business Media, LLC, part of Springer Nature.\nSummary::\n['KD', 'KD']\nMetastasis poses a significant challenge in combating tumors. Even in papillary thyroid cancer (PTC), which typically exhibits a favorable prognosis, high recurrence rates are attributed to metastasis. Cytoplasmic linker protein 170 (CLIP170) functions as a classical microtubule plus-end tracking protein (+TIP) and has shown close association with cell migration. Nevertheless, the specific impact of CLIP170 on PTC cells remains to be elucidated. Our analysis of the GEO and TCGA databases unveiled an association between CLIP170 and the progression of PTC. To explore the impact of CLIP170 on PTC cells, we conducted various assays. We evaluated its effects through CCK-8, wound healing assay, and transwell

In [21]:
import os

from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper

In [22]:
os.environ["SERP_API_KEY"] = ""
tool = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper())
tool.run("LLM Models")

ValidationError: 1 validation error for GoogleScholarAPIWrapper
__root__
  Did not find serp_api_key, please add an environment variable `SERP_API_KEY` which contains it, or pass `serp_api_key` as a named parameter. (type=value_error)

In [1]:
from langchain_community.retrievers import PubMedRetriever

In [2]:
retriever = PubMedRetriever()

In [3]:
retriever.invoke("chatgpt")

[Document(page_content="The internet, particularly social media, has become a popular resource for learning about health and investigating one's own health conditions. The development of AI chatbots has been fueled by the increasing availability of digital health data and advances in natural language processing techniques. While these chatbots are more accessible than before, they sometimes fail to provide accurate information. We used representative chatbots currently available (ChatGPT-3.5, Bing Chat, and Google Bard) to answer questions commonly asked by brain tumor patients. The simulated situations with questions were made and selected by the brain tumor committee. These questions are commonly asked by brain tumor patients. The goal of the study was introduced to each chatbot, the situation was explained, and questions were asked. All responses were collected without modification. The answers were shown to the committee members, and they were asked to judge the responses while bli

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
llm = Ollama(model="llama3")

In [4]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

KeyboardInterrupt: 

In [5]:
from langchain.tools import Tool
from langchain_community.retrievers import PubMedRetriever
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate


llm = Ollama(model="mistral", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")


retriever = PubMedRetriever()

# retriever.get_relevant_documents("chatgpt")

PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context. 
Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

# tag::qa[]
pubmed_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

prompt = "What is the latest research on COVID-19?"
response = pubmed_qa({"query": prompt})
# or pubmed_qa.invoke()

/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull mistral`.

In [1]:
from langchain.tools import Tool
from langchain_community.retrievers import PubMedRetriever
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate

llm = Ollama(model="llama3", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")

retriever = PubMedRetriever()

# Invoke the retriever with the query
retriever.get_relevant_documents("chatgpt")

PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context. 
Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

# tag::qa[]
pubmed_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

prompt = "What is the latest research on COVID-19?"
response = pubmed_qa({"query": prompt})
# or pubmed_qa.invoke()

/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
from langchain.tools import Tool
from langchain_community.retrievers import PubMedRetriever
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate


llm = Ollama(model="mistral", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")


retriever = PubMedRetriever()

# retriever.get_relevant_documents("chatgpt")

PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context. 
Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

# tag::qa[]
pubmed_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

prompt = "What is the latest research on COVID-19?"
response = pubmed_qa.invoke({"query": prompt})
# or pubmed_qa.invoke()

In [4]:
from langchain.tools import Tool
from langchain_community.retrievers import PubMedRetriever
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate


llm = Ollama(model="mistral", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")


retriever = PubMedRetriever()

# retriever.get_relevant_documents("chatgpt")

PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context. 
Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

# tag::qa[]
pubmed_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

prompt = "What is the latest research on COVID-19?"
#response = pubmed_qa({"query": prompt})
pubmed_qa.invoke(prompt)

{'query': 'What is the latest research on COVID-19?',
 'result': ' The latest research on COVID-19 and its impact on sexual and reproductive health is reviewed in this narrative article. The study explores how SARS-CoV-2 infection may affect menstrual cycles, assisted reproductive technology (ART) treatments, obstetric problems during pregnancy, and the psychological effects of family planning choices during the pandemic. New evidence suggests that SARS-CoV-2 may interfere with hormone synthesis and endometrial reactions, potentially impacting menstrual blood loss. ART results have not shown significant changes, but there is limited research on embryo quality. Obstetric problems are more likely when SARS-CoV-2 infection occurs during pregnancy, especially in the third trimester. The COVID-19 vaccine has minimal impact on menstrual cycles. Additionally, recent advancements in ACE2-related studies in pulmonary diseases are discussed, highlighting its potential role as a biomarker for ear

In [ ]:
from langchain.tools import Tool
from langchain_community.retrievers import PubMedRetriever
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate


llm = Ollama(model="llama3", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")


retriever = PubMedRetriever()

# retriever.get_relevant_documents("chatgpt")

PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context. 
Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

# tag::qa[]
pubmed_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

prompt = "Llamas are members of the camelid family."
#response = pubmed_qa({"query": prompt})
response = pubmed_qa.invoke(prompt)

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import VectorStoreRetriever

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
retriever = VectorStoreRetriever(vectorstore=vectorstore, embedding=embeddings)

print(retriever.embedding) # Prints the OpenAI embedding object used


ImportError: cannot import name 'VectorStoreRetriever' from 'langchain.retrievers' (/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain/retrievers/__init__.py)

In [7]:
import langchain
print(langchain.__version__)


0.1.20


In [8]:
from langchain.retrievers import VectorStoreRetriever


ImportError: cannot import name 'VectorStoreRetriever' from 'langchain.retrievers' (/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain/retrievers/__init__.py)

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import VectorIndexRetriever

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
retriever = VectorIndexRetriever(vectorstore=vectorstore, embedding=embeddings)

print(retriever.embedding_func) # Prints the embedding function used


ImportError: cannot import name 'VectorIndexRetriever' from 'langchain.retrievers' (/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain/retrievers/__init__.py)

In [10]:
from langchain.vectorstores import Chroma
from langchain.retrievers import VectorStoreRetriever

vectorstore = Chroma.from_documents(documents, embedding=embeddings)
retriever = VectorStoreRetriever(vectorstore=vectorstore, embedding=embeddings)


ImportError: cannot import name 'VectorStoreRetriever' from 'langchain.retrievers' (/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/langchain/retrievers/__init__.py)

In [11]:
from langchain.retrievers import PubMedRetriever

# Create a PubMedRetriever instance
retriever = PubMedRetriever(search_query="your search query", fetch_max=10)

# Access the embedding module used by the retriever
print(retriever.embedding_model)


AttributeError: 'PubMedRetriever' object has no attribute 'embedding_model'

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import PubMedRetriever

# Create a custom embedding model
embedding_model = HuggingFaceEmbeddings(model_name="your-huggingface-model")

# Create a PubMedRetriever instance with the custom embedding model
retriever = PubMedRetriever(search_query="your search query", fetch_max=10, embedding_model=embedding_model)

# Access the embedding module used by the retriever
print(retriever.embedding_model)


/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name sentence-transformers/your-huggingface-model. Creating a new one with MEAN pooling.
/home/melanie/anaconda3/envs/ba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: sentence-transformers/your-huggingface-model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`